In [9]:
# for converting straight to jpg. works on small files but crashes with large files

import numpy as np
import pathlib
from PIL import Image
import matplotlib.image
from aicspylibczi import CziFile
from pathlib import Path
import os
import tifffile
from tqdm import tqdm
import cv2

Image.MAX_IMAGE_PIXELS = None

# Path to directory containing the CZI files
# czi_dir = '/Users/mattzabriskie/Library/CloudStorage/Box-Box/MattZ_Work/Clinicians_PIs/LubdhaShah_ViolaRieke/cervical_fus_goat_research/Data/Histology/scanned_slides/2023.08.08'
czi_dir = '/Users/mzabriskie/Desktop/repos/czi_conversion/czi_files'

# # create directory to save tiff files
# tiff_dir = os.path.join(czi_dir, 'tiff_files')
# os.makedirs(tiff_dir, exist_ok=True)

# create directory to save jpg files
jpg_dir = os.path.join(czi_dir, 'jpg_files')
os.makedirs(jpg_dir, exist_ok=True)

# Get all files in directory ending with '.czi'
czi_files = [file for file in os.listdir(czi_dir) if file.endswith('.czi')]

# Loop over all files in the directory
for filename in tqdm(czi_files, desc="Converting files"):
    print(filename)

    # check to see if the CZI has already been converted to JPG
    if os.path.isfile(os.path.splitext(os.path.join(jpg_dir, filename))[0] + ".jpg"):
        print("A JPG file already exists for %s" % filename)

    # if no JPG, create JPG from CZI
    else:
        # Construct the full file path
        czi_file = os.path.join(czi_dir, filename)

        try:
            # Try to read the CZI file
            czi = CziFile(pathlib.Path(czi_file))

            czi_data = czi.read_mosaic(C=0, scale_factor=1)

        except Exception as e:
            print(f"File {filename} raised an error during processing: {str(e)}. Continuing with next file.")        
            continue
            
        # Construct the output file path (change the extension to .tiff)
        jpg_file = os.path.join(jpg_dir, os.path.splitext(filename)[0] + '.jpg')
        
        # Save as a TIFF file

        czi_data_resized = cv2.resize(czi_data[0], None, fx=0.1, fy=0.1)

        matplotlib.image.imsave(jpg_file, czi_data_resized)

print("All CZI files have been converted to JPG.")


Converting files: 100%|██████████| 2/2 [00:00<00:00, 265.89it/s]

2023_03_31__0060.czi
File 2023_03_31__0060.czi raised an error during processing: std::bad_alloc. Continuing with next file.
2023_03_31__0024.czi
File 2023_03_31__0024.czi raised an error during processing: std::bad_alloc. Continuing with next file.
All CZI files have been converted to JPG.


In [10]:
czi_files

['2023_03_31__0060.czi', '2023_03_31__0024.czi']

In [8]:
czi_data[0][500][0]

NameError: name 'czi_data' is not defined

In [44]:
from aicspylibczi import CziFile
import cv2
import numpy as np

try:
    # Open the CZI file
    czi = CziFile('/Users/mzabriskie/Desktop/repos/czi_conversion/czi_files/2023_03_31__0024.czi')
    # Read the data
    czi_data = czi.read_image()

    # Extract the image component
    image_data = czi_data[0][0][0][39]  # Select slice index 39

    # Convert to NumPy array
    image_np = np.array(image_data)

    # Resize the image to match the original CZI image dimensions
    original_height, original_width = 1200, 1600  # Update with original CZI image dimensions
    image_resized = cv2.resize(image_np, (original_width, original_height))

    # Save the resized image as a JPEG
    cv2.imwrite('output.jpg', cv2.cvtColor(image_resized, cv2.COLOR_RGB2BGR))  # Convert RGB to BGR for OpenCV

except Exception as e:
    print(f"An error occurred while opening or reading the CZI file: {e}")
    raise e  # Reraise the exception for further investigation


In [36]:
from aicspylibczi import CziFile
import numpy as np

try:
    # Open the CZI file
    czi = CziFile('/Users/mzabriskie/Desktop/repos/czi_conversion/czi_files/2023_03_31__0060.czi')
    # Read the data
    czi_data = czi.read_image()

    # Print the type and length of the CZI data
    print("Type of CZI data:", type(czi_data))
    print("Length of CZI data:", len(czi_data))

    # Print the shape of the first component
    print("Shape of component 0:", czi_data[0].shape)

    # Check if the second component is a list
    if isinstance(czi_data[1], list):
        # Print information about each element in the list
        for i, element in enumerate(czi_data[1]):
            print(f"Information about element {i}:", element)
    else:
        # Print the shape of the second component
        print("Shape of component 1:", czi_data[1].shape)

except Exception as e:
    print(f"An error occurred while opening or reading the CZI file: {e}")
    raise e  # Reraise the exception for further investigation


Type of CZI data: <class 'tuple'>
Length of CZI data: 2
Shape of component 0: (1, 1, 79, 1200, 1600, 3)
Information about element 0: ('S', 1)
Information about element 1: ('C', 1)
Information about element 2: ('M', 79)
Information about element 3: ('Y', 1200)
Information about element 4: ('X', 1600)
Information about element 5: ('A', 3)


In [47]:
from aicspylibczi import CziFile
import numpy as np
import cv2

try:
    # Open the CZI file
    czi = CziFile('/Users/mzabriskie/Desktop/repos/czi_conversion/czi_files/2023_03_31__0060.czi')
    # Read the data
    czi_data = czi.read_image()

    # Extract the image data from the first component
    image_data = czi_data[0]

    # Extract the dimensions and channels information from the second component
    dimensions_info = {}
    for element in czi_data[1]:
        dimensions_info[element[0]] = element[1]

    # Reshape the image data according to the dimensions information
    reshaped_image_data = np.reshape(image_data, tuple(dimensions_info[dim] for dim in ['S', 'M', 'Y', 'X', 'A']))

    # Optionally, extract specific channels if needed
    # For example, to extract the first channel:
    first_channel = reshaped_image_data[..., 0]

    # Save the processed image data as a JPG file
    cv2.imwrite('output2.jpg', image_data)

    print("JPG file exported successfully.")

except Exception as e:
    print(f"An error occurred while processing the CZI file: {e}")


JPG file exported successfully.
